<a href="https://colab.research.google.com/github/samyxandz/Ml-playgound/blob/main/Auto_Complete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Language Models: Auto-Complete


This Prototype uses N-grams, a simple but powerful method for language modeling.

#### RoadMap

<br>

1.   Load and preprocess data
Load and tokenize data.

*   Load and tokenize data.
*   Split the sentences into train and test sets.
*   Replace words with a low frequency by an unknown marker <unk>.


2.   Develop N-gram based language models


*   Compute the count of n-grams from a given data set.
*   Estimate the conditional probability of a next word with k-smoothing.


3. Use your own model to suggest an upcoming word given your sentence.



In [4]:
import math
import random
import numpy as np
import pandas as pd
import nltk
nltk.data.path.append('.')
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Load the data


In [5]:
with open("en_US.twitter.txt", "r") as f:
    data = f.read()

display(data[0:300])

"How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way too long.\nWhen you meet someone special... you'll know. Your heart will beat more rapidly and you'll smile for no reason.\nthey've decided its more fun if I don't.\nSo Tired D; Played Lazer Tag & Ran A "

## Pre-process the data

reprocess this data with the following steps:

1. Split data into sentences using "\n" as the delimiter.
2. Split each sentence into tokens.
3. Assign sentences into train or test sets.
4. Find tokens that appear at least N times in the training data.
5. Replace tokens that appear less than N times by <unk>

Split data into sentences

    Split data by linebreak "\n"
    Args:
        data: str
    Returns:
        A list of sentences


In [6]:
def split_to_sentences(data):

    sentences = data.split('\n')

    # - Remove leading and trailing spaces from each sentence
    # - Drop sentences if they are empty strings.
    sentences = [sentence.strip() for sentence in sentences]
    sentences = [s for s in sentences if len(s) > 0]

    return sentences

In [7]:
# test your code
x = """I have a pen.\nI have an apple. \nAh\nApple pen.\n"""
print(x)

split_to_sentences(x)

I have a pen.
I have an apple. 
Ah
Apple pen.



['I have a pen.', 'I have an apple.', 'Ah', 'Apple pen.']

Tokenize sentences (split a sentence into a list of words).

* Convert all tokens into lower case so that words which are capitalized (for example, at the start of a sentence) in the original text are treated the same as the lowercase versions of the words.
* Append each tokenized list of words into a list of tokenized sentences


    Tokenize sentences into tokens (words)
    
    Args:
        sentences: List of strings
    
    Returns:
        List of lists of tokens

In [8]:
def tokenize_sentences(sentences):

    tokenized_sentences = []


    for sentence in sentences:

        # Convert to lowercase letters
        sentence = sentence.lower()

        # Convert into a list of words
        tokenized = nltk.word_tokenize(sentence)

        # append the list of words to the list of lists
        tokenized_sentences.append(tokenized)

    return tokenized_sentences


In [9]:
sentences = ["Sky is blue.", "Leaves are green.", "Roses are red."]
tokenize_sentences(sentences)

[['sky', 'is', 'blue', '.'],
 ['leaves', 'are', 'green', '.'],
 ['roses', 'are', 'red', '.']]

#### Split into train and test sets

In [12]:
def get_tokenized_data(data):
    sentences = split_to_sentences(data)
    tokenized_sentences = tokenize_sentences(sentences)
    return tokenized_sentences

In [13]:
tokenized_data = get_tokenized_data(data)
random.seed(87)
random.shuffle(tokenized_data)

train_size = int(len(tokenized_data) * 0.8)
train_data = tokenized_data[0:train_size]
test_data = tokenized_data[train_size:]

### Making the dictionary

* focus on the words that appear at least N times in the data.



In [14]:
def count_words(tokenized_sentences):

    word_counts = {}

    for sentence in tokenized_sentences:

        for token in sentence:
            word_counts[token] = word_counts.get(token, 0) + 1

    return word_counts

In [15]:

tokenized_sentences = [['sky', 'is', 'blue', '.'],
                       ['leaves', 'are', 'green', '.'],
                       ['roses', 'are', 'red', '.']]
count_words(tokenized_sentences)

{'sky': 1,
 'is': 1,
 'blue': 1,
 '.': 3,
 'leaves': 1,
 'are': 2,
 'green': 1,
 'roses': 1,
 'red': 1}

Handling 'Out of Vocabulary' words

To handle unknown words during prediction, use a special token to represent all unknown words 'unk'.

* Modify the training data so that it has some 'unknown' words to train on.
* Words to convert into "unknown" words are those that do not occur very frequently in the training set.
* Create a list of the most frequent words in the training set, called the closed vocabulary .
* Convert all the other words that are not part of the closed vocabulary to the token 'unk'.

In [16]:
def get_words_with_nplus_frequency(tokenized_sentences, count_threshold):

    closed_vocab = []
    word_counts = count_words(tokenized_sentences)
    # for each word and its count
    for word, count in word_counts.items():
        if count >= count_threshold:
            # append the word to the list
            closed_vocab.append(word)


    return closed_vocab

The words that appear count_threshold times or more are in the closed vocabulary.

* All other words are regarded as unknown.
* Replace words not in the closed vocabulary with the token $<unk>$.

In [17]:
def replace_oov_words_by_unk(tokenized_sentences, vocabulary, unknown_token='<unk>'):

    vocabulary = set(vocabulary)

    # Initialize a list that will hold the sentences
    # after less frequent words are replaced by the unknown token
    replaced_tokenized_sentences = []


    for sentence in tokenized_sentences:

        # Initialize the list that will contain
        # a single sentence with "unknown_token" replacements
        replaced_sentence = []
        # for each token in the sentence
        for token in sentence:
            replaced_sentence.append(token if token in vocabulary else unknown_token)

        replaced_tokenized_sentences.append(replaced_sentence)

    return replaced_tokenized_sentences

In [18]:
def preprocess_data(train_data, test_data, count_threshold):


    # Get the closed vocabulary using the train data
    vocabulary = get_words_with_nplus_frequency(train_data, count_threshold)

    train_data_replaced = replace_oov_words_by_unk(train_data, vocabulary)
    test_data_replaced = replace_oov_words_by_unk(test_data, vocabulary)

    return train_data_replaced, test_data_replaced, vocabulary

## Preprocess the train and test data

In [19]:
minimum_freq = 2
train_data_processed, test_data_processed, vocabulary = preprocess_data(train_data, test_data, minimum_freq)

## Developing the  n-gram based language

Asumptions

* Assume the probability of the next word depends only on the previous n-gram.
* The previous n-gram is the series of the previous 'n' words


The conditional probability for the word at position 't' in the sentence, given that the words preceding it are $$w_{t-1}, w_{t-2} \cdots w_{t-n}$$ is:
$P(w_t | w_{t-1}\dots w_{t-n})$
  

- The probability can be estimated as a ratio, where
- The numerator is the number of times word 't' appears after words t-1 through t-n appear in the training data.
- The denominator is the number of times word t-1 through t-n appears in the training data.

    $$ \hat{P}(w_t | w_{t-1}\\dots w_{t-n}) = \\frac{C(w_{t-1}\\dots w_{t-n}, w_n)}{C(w_{t-1}\\dots w_{t-n})}  $$


The function $C(\cdots)$ denotes the number of occurence of the given sequence.
$\hat{P}$ means the estimation of $P$.

  The equation tells us that to estimate probabilities based on n-grams, you need the counts of n-grams (for denominator) and (n+1)-grams (for numerator).

 ### Count The N-Grams

 function that computes the counts of n-grams for an arbitrary number $n$

 When computing the counts for n-grams, prepare the sentence beforehand by prepending $n-1$ starting markers $<s>$ to indicate the beginning of the sentence.

In [20]:
def count_n_grams(data, n, start_token='<s>', end_token='<e>'):

    n_grams = {}

    for sentence in data:
        sentence = [start_token] * n + sentence + [end_token]

        sentence = tuple(sentence)

        for i in range(len(sentence) - n + 1):
            n_gram = sentence[i : i + n]
            n_grams[n_gram] = n_grams.get(n_gram, 0) + 1

    return n_grams

In [21]:

sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
print("Uni-gram:")
print(count_n_grams(sentences, 1))
print("Bi-gram:")
print(count_n_grams(sentences, 2))

Uni-gram:
{('<s>',): 2, ('i',): 1, ('like',): 2, ('a',): 2, ('cat',): 2, ('<e>',): 2, ('this',): 1, ('dog',): 1, ('is',): 1}
Bi-gram:
{('<s>', '<s>'): 2, ('<s>', 'i'): 1, ('i', 'like'): 1, ('like', 'a'): 2, ('a', 'cat'): 2, ('cat', '<e>'): 2, ('<s>', 'this'): 1, ('this', 'dog'): 1, ('dog', 'is'): 1, ('is', 'like'): 1}


### Estimating probability

we  estimate the probability of a word given the prior 'n' words using the n-gram counts.

$ \hat{P}(w_t | w_{t-1}\dots w_{t-n}) = \frac{C(w_{t-1}\dots w_{t-n}, w_n)}{C(w_{t-1}\dots w_{t-n})} $
    
This formula doesn't work when a count of an n-gram is zero..
  
  - Suppose we encounter an n-gram that did not occur in the training data.
  - Then, the equation cannot be evaluated (it becomes zero divided by zero).
  A way to handle zero counts is to add k-smoothing.
  
  - K-smoothing adds a positive constant $k$ to each numerator and $k \times |V|$ in the denominator, where $|V|$ is the number of words in the vocabulary.
    
  $ \hat{P}(w_t | w_{t-1}\dots w_{t-n}) = \frac{C(w_{t-1}\dots w_{t-n}, w_n) + k}{C(w_{t-1}\dots w_{t-n}) + k|V|}  $

  For n-grams that have a zero count, the equation  becomes $\\frac{1}{|V|}$
  - This means that any n-gram with zero count has the same probability of $\\frac{1}{|V|}$

In [22]:
def estimate_probability(word, previous_n_gram,
                         n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):


    previous_n_gram = tuple(previous_n_gram)

    previous_n_gram_count = n_gram_counts.get(previous_n_gram, 0)

    # apply k-smoothing
    denominator = previous_n_gram_count + k * vocabulary_size

    # Define n plus 1 gram as the previous n-gram plus the current word as a tuple
    n_plus1_gram = previous_n_gram + (word,)

    # Set the count to the count in the dictionary,
    # otherwise 0 if not in the dictionary
    n_plus1_gram_count = n_plus1_gram_counts.get(n_plus1_gram, 0)

    # Define the numerator  apply smoothing
    numerator = n_plus1_gram_count + k

    # Calculate the probability as the numerator divided by denominator
    probability = numerator / denominator

    ### END CODE HERE ###

    return probability

In [23]:
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)
tmp_prob = estimate_probability("cat", "a", unigram_counts, bigram_counts, len(unique_words), k=1)

print(f"The estimated probability of word 'cat' given the previous n-gram 'a' is: {tmp_prob:.4f}")

The estimated probability of word 'cat' given the previous n-gram 'a' is: 0.3333


### Estimate probabilities for all words

The function defined below loops over all words in vocabulary to calculate probabilities for all possible words.

In [24]:
def estimate_probabilities(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0):

    previous_n_gram = tuple(previous_n_gram)

    vocabulary = vocabulary + ['<e>', '<unk>']
    vocabulary_size = len(vocabulary)

    probabilities = {}
    for word in vocabulary:
        probability = estimate_probability(word, previous_n_gram,
                                           n_gram_counts, n_plus1_gram_counts,
                                           vocabulary_size, k=k)
        probabilities[word] = probability

    return probabilities

In [25]:
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))
unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)
estimate_probabilities("a", unigram_counts, bigram_counts, unique_words, k=1)

{'is': 0.09090909090909091,
 'this': 0.09090909090909091,
 'dog': 0.09090909090909091,
 'like': 0.09090909090909091,
 'a': 0.09090909090909091,
 'i': 0.09090909090909091,
 'cat': 0.2727272727272727,
 '<e>': 0.09090909090909091,
 '<unk>': 0.09090909090909091}

### Count and probability matrices



In [26]:
def make_count_matrix(n_plus1_gram_counts, vocabulary):
    vocabulary = vocabulary + ["<e>", "<unk>"]

    # obtain unique n-grams
    n_grams = []
    for n_plus1_gram in n_plus1_gram_counts.keys():
        n_gram = n_plus1_gram[0:-1]
        n_grams.append(n_gram)
    n_grams = list(set(n_grams))

    # mapping from n-gram to row
    row_index = {n_gram:i for i, n_gram in enumerate(n_grams)}
    # mapping from next word to column
    col_index = {word:j for j, word in enumerate(vocabulary)}

    nrow = len(n_grams)
    ncol = len(vocabulary)
    count_matrix = np.zeros((nrow, ncol))

    for n_plus1_gram, count in n_plus1_gram_counts.items():
        n_gram = n_plus1_gram[0:-1]
        word = n_plus1_gram[-1]
        if word not in vocabulary:
            continue
        i = row_index[n_gram]
        j = col_index[word]
        count_matrix[i, j] = count

    count_matrix = pd.DataFrame(count_matrix, index=n_grams, columns=vocabulary)
    return count_matrix

this function calculates the probabilities of each word given the previous n-gram, and stores this in matrix form

In [27]:
def make_probability_matrix(n_plus1_gram_counts, vocabulary, k):
    count_matrix = make_count_matrix(n_plus1_gram_counts, unique_words)
    count_matrix += k
    prob_matrix = count_matrix.div(count_matrix.sum(axis=1), axis=0)
    return prob_matrix

##  Building the system auto-complete system


In [28]:
def suggest_a_word(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0, start_with=None):


    # length of previous words
    n = len(list(n_gram_counts.keys())[0])

    previous_n_gram = previous_tokens[-n:]

    # Estimate the probabilities that each word in the vocabulary
    probabilities = estimate_probabilities(previous_n_gram,
                                           n_gram_counts, n_plus1_gram_counts,
                                           vocabulary, k=k)

    # Initialize suggested word to None
    suggestion = None
    # Initialize the highest word probability to 0
    max_prob = 0

    # For each word and its probability in the probabilities dictionary:
    for word, prob in probabilities.items():
        if start_with:
            if not word.startswith(start_with):
                continue

        # Check if this word's probability
        # is greater than the current maximum probability
        if prob > max_prob:
            suggestion = word
            max_prob = prob

    return suggestion, max_prob

In [29]:
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)

previous_tokens = ["i", "like"]
tmp_suggest1 = suggest_a_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k=1.0)
print(f"The previous words are 'i like',\n\tand the suggested word is `{tmp_suggest1[0]}` with a probability of {tmp_suggest1[1]:.4f}")

print()

tmp_starts_with = 'c'
tmp_suggest2 = suggest_a_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k=1.0, start_with=tmp_starts_with)
print(f"The previous words are 'i like', the suggestion must start with `{tmp_starts_with}`\n\tand the suggested word is `{tmp_suggest2[0]}` with a probability of {tmp_suggest2[1]:.4f}")


The previous words are 'i like',
	and the suggested word is `a` with a probability of 0.2727

The previous words are 'i like', the suggestion must start with `c`
	and the suggested word is `cat` with a probability of 0.0909


### multiple suggestions

loop over various n-gram models to get multiple suggestions.

In [30]:
def get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with=None):
    model_counts = len(n_gram_counts_list)
    suggestions = []
    for i in range(model_counts-1):
        n_gram_counts = n_gram_counts_list[i]
        n_plus1_gram_counts = n_gram_counts_list[i+1]

        suggestion = suggest_a_word(previous_tokens, n_gram_counts,
                                    n_plus1_gram_counts, vocabulary,
                                    k=k, start_with=start_with)
        suggestions.append(suggestion)
    return suggestions

## Final Tests

In [32]:
n_gram_counts_list = []
for n in range(1, 6):

    n_model_counts = count_n_grams(train_data_processed, n)
    n_gram_counts_list.append(n_model_counts)

In [33]:
previous_tokens = ["this", 'is']
tmp_suggest = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with=None)

print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest)

The previous words are ['this', 'is'], the suggestions are:


[('the', 0.020172225129644606),
 ('the', 0.0022552401167418414),
 ('i', 6.745362563237774e-05),
 ('i', 6.745362563237774e-05)]